In [147]:
%pylab inline
from glob import glob
import math
from PIL import Image
import json
import sklearn.manifold
import os

def resize_to_power_of_two(nSize):
    pos =  math.ceil(math.log2(nSize))  #(ceiling of log n with base 2)
    p   = pow(2, pos) 
    return p

def generate_spritesheet(photoSize, imageDir, outDir, game, corpus, jsonFile): 
    """\
    Make a contact sheet from a group of filenames:
    fnames       A list of names of the image files  
    ncols        Number of columns in the contact sheet
    nrows        Number of rows in the contact sheet
    photow       The width of the photo thumbs in pixels
    photoh       The height of the photo thumbs in pixels
    returns a PIL image object.
    """  
    images = glob(imageDir + '/*.png')
    images = sorted(images, key=lambda x: int(os.path.basename(x)[:-4]))   
    spritesheet_name = outDir + game + '/' + corpus
    print(spritesheet_name)
    

    # Read current json
    f = open(jsonFile, 'r+')
    corporaDict = json.load(f)
    if corpus not in corporaDict:
        corporaDict[corpus] = {}
    
    # Calculate the size of the output image, based on the photo thumb sizes
    
    totalNum = len(images)
    print(len(images))
    
    ncols = 64
    nrows = math.ceil(totalNum/ncols)
    
    photow = photoSize[0]
    photoh = photoSize[1]
   
    
    imgWidth = ncols*photow
    imgHeight = nrows*photoh
    
    imgWidth_resized = resize_to_power_of_two(imgWidth)
    imgHeight_resized = resize_to_power_of_two(imgHeight)
    
    isize = (imgWidth_resized,imgHeight_resized) # adapt to THREE
    print('imageSize=',isize, "photow", photow, "photoh", photoh, "")

    # Create the new image. The background doesn't have to be white
    white = (0,0,0)
    inew = Image.new('RGB',isize,white)
    
    # Create JSON file to store the UV position of each image
    spriteDict = {}
    spriteDict['game'] = game
    spriteDict['corpus'] = corpus
    spriteDict['rows'] = nrows
    spriteDict['columns'] = ncols
    spriteDict['totalCount'] = totalNum
    spriteDict['spriteWidth'] = photow
    spriteDict['spriteHeight'] = photoh 
    
    count = 0
    # Insert each thumb:
    for irow in range(nrows):
        for icol in range(ncols):
            left = icol*(photow)
            right = left + photow
            upper = irow*(photoh)
            lower = upper + photoh
            bbox = (left,upper,right,lower)
            try:
                # Read in an image and resize appropriately
                img = Image.open(images[count]).resize((photow,photoh))
            except:
                break
            inew.paste(img,bbox)
            if count not in spriteDict.keys():
                spriteDict[count] = {}
            spriteDict[count]['filename'] = os.path.basename(images[count])
            spriteDict[count]['uvOffset_u'] = left/imgWidth
            spriteDict[count]['uvOffset_v'] = 1.0-lower/imgHeight_resized 
            spriteDict[count]['uvRepeat_u'] = photow/imgWidth_resized
            spriteDict[count]['uvRepeat_v'] = photoh/imgHeight_resized
            count += 1 
            
    inew.save(spritesheet_name + '.png')
    
    #add new content
    corporaDict[corpus] = spriteDict
    js = json.dumps(corporaDict, indent=2)
    f.seek(0) 
    f.write(js)
    f.close()
    
    return inew

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [150]:
def generate_positions(embeddingDir, outputDir, dimensions, perplexity, game, corpus, jsonFile):
    embeddings = glob(embeddingDir + '/*.npy')
    embeddings = sorted(embeddings, key=lambda x: int(os.path.basename(x)[:-4]))
    length = len(embeddings)
    states = zeros((length, 256))
    for i in range(len(states)):
        states[i] = load(embeddings[i])

    tsne = sklearn.manifold.TSNE(n_components = dimensions, perplexity = perplexity)
    positions = tsne.fit_transform(states)
    
    positionbuffer_name = 'None'
    if positions is not None:
        positionbuffer_name = outDir + game + '/' + corpus + '.bin'
        positions.tofile(positionbuffer_name) #default is binary write
        dimensions = positions.shape[1]
    else:
        dimensions = 0
        
    # Read current json
    f = open(jsonFile, 'r+')
    corporaDict = json.load(f)
    if corpus not in corporaDict:
        corporaDict[corpus] = {}
    corporaDict[corpus]['dimensions'] = dimensions
    corporaDict[corpus]['perplexity'] = perplexity
    corporaDict[corpus]['bin'] = positionbuffer_name
    
    js = json.dumps(corporaDict, indent=2) 
    f.seek(0) 
    f.write(js)
    f.close()
    

In [ ]:
######Parameters Begin#########
dimensions = 3
perplexity = 30
game = 'Super Mario World (USA)'
gamedir = '../Game_data/Super Mario World (USA)_new/'
runTSNE = True
runSpritesheet = False
dir_screenshots = 'screenshots'
dir_embeddings = 'states'
photow,photoh = 64,56
photoSize = (64,56)
outputDir = './pic/'
######Parameters End###########

#Find corpora
corpora = {}
corpora['dir'] = gamedir
corpora['game'] = game

#Create json file
jsonFile = outDir + game + '.json'
f = open(jsonFile, 'w+')
js = json.dumps(corpora, indent=2)
f.write(js)
f.close()

folders = glob(gamedir + '*')
for i in range(len(folders)):
    folder = folders[i]
    idx = folder.rfind('/')
    key = folder[idx+1:]
    corpora[key] = folder
    corpus = key
    if runSpritesheet:
        imageDir = folder + '/'+ dir_screenshots
        inew = generate_spritesheet(photoSize, imageDir, outputDir, game, corpus, jsonFile)
        inew.show()

    if runTSNE:
        embeddingDir = folder + '/' + dir_embeddings
        generate_positions(embeddingDir, outputDir, dimensions, perplexity, game, corpus, jsonFile)
          

In [88]:
f = open('a.json', 'r+')
data = json.load(f)
print(data)
data['1'] = 2
data['2'] = 3
print(data)
#add new content
js = json.dumps(data, indent=2)

f.seek(0) 
f.write(js)
f.close()

{'0': 0, '1': 1, '2': 2}
{'0': 0, '1': 2, '2': 3}
